In [1]:
from ground_truth import filter_profiles, renormalize_profile, SpeciesLabel
from chronostrain import extract_chronostrain_prediction, chronostrain_prediction_to_profile, ChronoStrainInferenceResult

ImportError: attempted relative import with no known parent package

In [2]:
sample_tag, profile_df = next(iter(filter_profiles("../../data/gs_strain_madness_short_long.profile.txt")))

abundance_profile = renormalize_profile(profile_df, SpeciesLabel("Escherichia", "coli"))

In [8]:
display(abundance_profile.abundance_ratios)
display(abundance_profile.strain_ids)

array([3.13227899e-05, 3.33160583e-04, 2.44602514e-02, 9.22598539e-04,
       6.26455798e-05, 2.96996999e-03, 9.57338360e-03, 6.89101378e-04,
       1.08718556e-02, 2.51436577e-03, 1.30986212e-04, 5.41030007e-05,
       1.36966018e-03, 2.47421565e-02, 1.08206001e-03, 3.41526616e-01,
       7.57442010e-04, 8.58244443e-03, 5.73320956e-02, 3.13227899e-05,
       1.16509388e-01, 8.21511353e-03, 2.22107056e-04, 4.08904784e-03,
       8.82733170e-05, 1.96479318e-04, 1.19596107e-04, 4.72689375e-04,
       2.52290835e-03, 3.53093268e-04, 1.13046796e-03, 1.53766423e-02,
       8.08697484e-04, 8.99818328e-04, 4.47061638e-04, 3.47398215e-04,
       2.27802108e-05, 2.05021897e-04, 1.11395231e-02, 1.01884493e-02,
       1.08206001e-03, 3.41703162e-05, 4.47061638e-03, 8.14392537e-04,
       8.45715327e-04, 8.28630169e-04, 1.08206001e-04, 3.54232278e-03,
       5.69505271e-06, 3.04685320e-04, 4.32254501e-03, 4.27128953e-05,
       2.34892449e-02, 2.10289821e-02, 9.39683697e-05, 1.39528791e-04,
      

['spades_And10062_S12_L001',
 'spades_ESBL10056_S9_L001',
 'spades_And5213_S1_L001',
 'spades_And9855_S12_L001',
 'spades_ESBL7021_S2_L001',
 'spades_And4191_S5_L001',
 'spades_And4355_S8_L001',
 'spades_And5042_S4_L001',
 'spades_And4180_S6_L001',
 'spades_ESBL5734_S3_L001',
 'spades_And7051_S7_L001',
 'spades_ESBL7348_S6_L001',
 'spades_AND9183_S11_L001',
 'spades_ESBL5402_S6_L001',
 'spades_ESBL7531_S9_L001',
 'spades_And7556_S8_L001',
 'spades_And9426_S4_L001',
 'spades_ESBL10030_S2_L001',
 'spades_ESBL9044_S9_L001',
 'spades_And10073_S10_L001',
 'spades_ESBL9861_S4_L001',
 'spades_ESBL7534_S10_L001',
 'spades_And4366_S2_L001',
 'spades_And4902_S2_L001',
 'spades_ESBL6455_S6_L001',
 'spades_ESBL9336_S1_L001',
 'spades_ESBL7359_S3_L001',
 'spades_ESBL1009_S1_L001',
 'spades_And9454_S3_L001',
 'spades_ESBL6364_S9_L001',
 'spades_And4955_S10_L001',
 'spades_ESBL7532_S2_L001',
 'spades_ESBL5621_S1_L001',
 'spades_ESBL5792_S6_L001',
 'spades_ESBL9166_S11_L001',
 'spades_AND4471_S1_L001'